In [ ]:
module Fire

using LinearAlgebra, SignedDistanceFields, Contour, HDF5

# create the mesh and the initial flame front
const dx = 0.01
const xrange = -6:dx:6
const yrange = -6:dx:6

const mesh = [(x,y) for x=xrange, y=yrange]

const ϕ0 = [norm(p) - 0.1 for p in mesh];

# specify all the mountains
const mountains = [
    (1.5,    2.000,  3.000, 1.000), 
    (2.000,  1.000, -1.000, 1.500),
    (1.000, -2.000,  3.000, 1.000)
]

elevation(x, y) = sum(m[1]*exp(-((x-m[2])^2+(y-m[3])^2)/(2*m[4]^2)) for m in mountains)

# analytic derivative of the slope
slope(x,y) = sum( 
    (m[1]/m[4]^2) * sqrt( exp(- ((x-m[2])^2 + (y-m[3])^2) / m[4]^2 ) * ( (x-m[2])^2 + (y-m[3])^2 ) )
    for m in mountains
)



function compute_rate_of_spread(R0=0.2, wind=[0.,0.])
    
    ϕs = [25*slope(p[1], p[2])^2 for p in mesh]
    
    ϕw = [0.0 for p in mesh]
    
    S = R0 * (1.0 .+ ϕw + ϕs)
    
    return S
    
end

const _S = compute_rate_of_spread()


function level_set_step!(ϕ, dt, S)
    
    # propagate
    ϕ .-= dt * S
    
    # recompute level set
    bool_ϕ = [p <= 0.0 for p in ϕ]
    sdf_ϕ = sdf(bool_ϕ)
    ϕ .= sdf_ϕ * dx
    
    return ϕ
end



function level_set_step!(ϕ, dt)
    
    ϕ = level_set_step!(ϕ, dt, _S)
    
    return ϕ
end


function simulate!(ϕ0, dt, steps)

    ϕ = copy(ϕ0)
    ϕ_history = [copy(ϕ)]
    
    for i=1:steps
        level_set_step!(ϕ, dt)
        push!(ϕ_history, copy(ϕ))
    end
    return ϕ_history
end



function get_index(pos)
    xi = findmin(abs.(xrange .- pos[1]))[2]
    yi = findmin(abs.(yrange .- pos[2]))[2]
    return xi, yi
end

function get_normal(pos, ϕ)
    
    xi, yi = get_index(pos)
    
    xmax, ymax = size(ϕ)
    
    xp = min(xi+1, xmax)
    xm = max(xi-1, 1)

    yp = min(yi+1, ymax)
    ym = max(yi-1, 1)

    
    dphidx =  (ϕ[xp, yi] - ϕ[xm, yi])
    dphidy =  (ϕ[xi, yp] - ϕ[xi, ym]) 
    
    n = normalize([dphidx, dphidy])
   
    return n
end

function get_tangent(pos, ϕ)
    
    n = get_normal(pos, ϕ)
    
    return [-n[2], n[1]]
end


function wrapTo2Pi(angle)
    θ = atan(sin(angle), cos(angle))
    return θ < 0 ? 2π + θ : θ
end


 
function sim_and_save(filename, ϕ0, dt, steps)
    ϕ = copy(ϕ0)
    ts = collect(range(;start = 0.0, step=dt, length=steps))
    
    #write times
    h5write(filename, "phi/ts", collect(ts))

    
    for (i, t) in enumerate(ts)
        println("[SIM] t: $t")
        h5write(filename, "phi/phi$i", ϕ)        
        Fire.level_set_step!(ϕ, dt)
    end
    
    
    # write contours
    for (i, t) in enumerate(ts)
        _ϕ = get_ϕ_file(t, filename)
        xs, ys = get_contours(_ϕ)
        h5write(filename, "phi/contour_x$i", xs)
        h5write(filename, "phi/contour_y$i", ys)
    end
    
end

function get_contours(ϕ)
    
    c = Contour.contour(xrange, yrange, ϕ, 0.0)
    
    xs = Float64[]
    ys = Float64[]

    for l in c.lines
        _xs, _ys = coordinates(l)
        xs = vcat(xs, _xs, [NaN])
        ys = vcat(ys, _ys, [NaN])
    end

    return xs, ys
end


const saved_sim_file_1 = "data/phi_0R1.h5"
const saved_sim_file_2 = "data/phi_0R01.h5"

# create a function to load sim at some time
function get_ϕ_file(t, filename=saved_sim_file_2)

    saved_ts = h5read(filename, "phi/ts")    
    indA = findlast(saved_ts .<= t) 
    indB = indA+1
    
    tA = saved_ts[indA]
    tB = saved_ts[indB]
    
    @assert !isnothing(indA)
    
    ϕA = h5read(filename, "phi/phi$(indA)")
    ϕB = h5read(filename, "phi/phi$(indB)")
    
    frac = (t-tA)/(tB-tA)
    
    ϕ = (1-frac) * ϕA + frac * ϕB
    
    return ϕ
end




# create a function to load sim at some time
function get_ϕ_cont(t, filename=saved_sim_file_2)

    saved_ts = h5read(filename, "phi/ts")    
    ind = findlast(saved_ts .<= t) 
    
    @assert !isnothing(ind)
    
    xs = h5read(filename, "phi/contour_x$(ind)")
    ys = h5read(filename, "phi/contour_y$(ind)")
    return xs, ys
end







if !isfile(saved_sim_file_1) || !isfile(saved_sim_file_2)
    
    println("Data file doesnt exist - generating now...")
    sim_and_save(saved_sim_file_1, ϕ0, 0.1, 10)
    
    # initialize the finer sim at the result of t=0.6
    ϕ7 = get_ϕ_file(0.601, saved_sim_file_1)
    
    sim_and_save(saved_sim_file_2, ϕ7, 0.01, 100)
    println("Data files generated!")
end


# LOAD TS, Φs
const TS = h5read(saved_sim_file_2, "phi/ts")
const Φs = [h5read(saved_sim_file_2, "phi/phi$(ind)") for ind=1:length(TS)]

function get_ϕ(t)

    indA = findlast(TS .<= t) 
    indB = indA+1
    
    tA = TS[indA]
    tB = TS[indB]
    
    @assert !isnothing(indA)
    
    ϕA = Φs[indA]
    ϕB = Φs[indB]
    
    frac = (t-tA)/(tB-tA)
    
    ϕ = (1-frac) * ϕA + frac * ϕB
    
    return ϕ
end
    


    
end

In [ ]:
do_plots = false

In [ ]:
using Plots, LinearAlgebra, .Fire, BenchmarkTools, DifferentialEquations, HDF5, NLopt

In [ ]:
@time Fire.get_ϕ(0.0);
@btime Fire.get_ϕ(3.0/3600);

In [ ]:
contourf(Fire.xrange, Fire.yrange, Fire.compute_rate_of_spread()')

In [ ]:
@time begin plot()
ts = 0:0.1:1.0
for t in ts
#      contour!(Fire.xrange, Fire.yrange, Fire.get_ϕ(t)', levels=[0.0])
    xs, ys = Fire.get_ϕ_cont(t)
    plot!(xs, ys, label=false)
    xlims!(-6,6)
    ylims!(-6,6)
end
plot!()
end

In [ ]:
heli_pos0 = [0, -3.00]

In [ ]:
function sat(x; l=20.0)
    
    if norm(x) >= l
        return l * x / norm(x)
    else
        return x
    end
end

function dynamics(state, u)
    
    pos = state[1:2]   # m
    θ   = state[3]     # rad
    v   = state[4]     # m/s
    
    
    acc = u[1]         # m/s^2
    #roll_rate = u[2]
    roll_angle = u[2]  # rad
    
    # turn radius and heading turn rate
    g = 9.81
    # r = v^2 / (g * tan(roll_angle))
    ω = g * tan(roll_angle) / v # = v/r
    
    # return
    xdot = v*cos(θ)
    ydot = v*sin(θ)
    θdot = ω
    vdot = acc
    
    return [xdot, ydot, θdot, vdot]
end

function disc_dynamics(state, u, dt)
    
    x = dt * dynamics(state, u)
    
    x[3] = atan(sin(x[3]), cos(x[3]))
    
    return x
end

##  Simulate the nominal controller

In [ ]:
function π_nominal(state, ϕ; offset=0.1)
    
    pos = state[1:2]   # m
    θ   = state[3]     # rad
    v   = state[4]     # m/s

    # current velocity in world frame
    vel = [v*cos(θ), v*sin(θ)]
    
    pos_km = pos/1000.0
    
    # get the current distance to the fire
    heli_ϕ = ϕ[Fire.get_index(pos_km)...]
    
    # get the local normal and tangent vectors to the fire front
    normal_vec = Fire.get_normal(pos_km, ϕ)
    tangent_vec = Fire.get_tangent(pos_km, ϕ)
    
    # get the desired velocity vector, if the vehicle was a double integrator
    des_vel = 15.0*tangent_vec + 8.0 * normal_vec * (offset - heli_ϕ)
    
    # p controller to get linear acceleration
    kp = 1.0/30.0
    des_acc = kp * (dot(des_vel, vel)/ norm(vel) - norm(vel))
    
    
    # compute heading error
    vel3 = [vel[1], vel[2], 0.0]
    des_vel3 = [des_vel[1], des_vel[2], 0.0]
    heading_error = (cross(vel3, des_vel3) / norm(vel3) / norm(des_vel3))[3]
    
    # get the desired roll
    kr = 1.0/7.0
    des_roll = kr * heading_error
    
    return [des_acc, des_roll]
    
end

In [ ]:
function closed_loop_nominal(state,params,t)

    t_hour = t / 3600.0

    ϕ = Fire.get_ϕ(t_hour)

    u = π_nominal(state, ϕ)

    return dynamics(state, u)
end

In [ ]:
function simulate_nominal(state0, t_max)
    
    tspan = (0, t_max)
    
    prob = ODEProblem(closed_loop_nominal, state0, tspan)
    
    sol = solve(prob, abstol=1e-2, reltol=1e-2, dt=10.0)
    
    return sol
end

In [ ]:
state0 = [0, -2.5*1000, 0, 15.0]
ϕ0 = Fire.get_ϕ(0.0);# Fire.simulate!(Fire.ϕ0, 0.1, 7)[end]

In [ ]:
π_nominal(state0, ϕ0)

In [ ]:
# precompile
simulate_nominal(state0, 20);

In [ ]:
sol = simulate_nominal(state0, 50*60)

In [ ]:
struct Trajectory
    ts::Vector{Float64}
    xs::Vector{Vector{Float64}}
    us::Vector{Vector{Float64}}
end


Trajectory() = Trajectory([],[],[])

## simulate the standard approach - backup CBFs

In [ ]:
const max_spread_rate = 8.0

function π_backup(state, ϕ)
    
    pos = state[1:2]   # m
    θ   = state[3]     # rad
    v   = state[4]     # m/s
    
    pos_km = pos/1000.0
    vel = [v*cos(θ), v*sin(θ)]
    
    # get the desired velocity
    des_vel = 15.0*Fire.get_normal(pos_km, ϕ)
    
    # p controller to get linear acceleration
    kp = 1.0/30.0
    des_acc = kp * (dot(des_vel, vel)/ norm(vel) - norm(vel))
    
    # compute heading error
    vel3 = [vel[1], vel[2], 0.0]
    des_vel3 = [des_vel[1], des_vel[2], 0.0]
    heading_error = (cross(vel3, des_vel3) / norm(vel3) / norm(des_vel3))[3]
    
    # get the desired roll
    kr = 1.0/7.0
    des_roll = kr * heading_error
    
    return [des_acc, des_roll]
    
end

function backup_dynamics(state, ϕ0, t)
    
    ϕ = ϕ0 .- max_spread_rate * (t/3600)
    
    u = π_backup(state, ϕ)
    
    return dynamics(state, u)
    
end

function backup_min_h(state0, ϕ0, predict_horizon)
    
    prob = ODEProblem(backup_dynamics, state0, (0.0, predict_horizon), ϕ0)
    
    sol = solve(prob, Euler(), dt=10.0) # solve the forward dynamics
    
    min_h = ϕ0[Fire.get_index(sol(0.0)/1000)...]
    
    for t=0:5.0:predict_horizon
        min_h = min(min_h, ϕ0[Fire.get_index(sol(t)/1000)...] - max_spread_rate * (t/3600))
    end
    
    return min_h
end

function π_safe(state, ϕ)
    
    min_h = backup_min_h(state, ϕ, 120.0)
    
    λ = exp(- max(0.0, min_h)/0.5)
    
    u = λ * π_backup(state, ϕ) + (1-λ)*π_nominal(state, ϕ)
    
    return u, λ
end

In [ ]:
function closed_loop_backup_safety(state, params, t)
    
    ϕ = Fire.get_ϕ(t / 3600.0)
    
    u, λ = π_safe(state, ϕ)
    
    return dynamics(state, u)
end

In [ ]:
function simulate_backup_safety(state0, t_max)
    
    tspan = (0.0, t_max)
    
    prob = ODEProblem(closed_loop_backup_safety, state0, tspan)
    
    sol = solve(prob, abstol=1e-2, reltol=1e-2, dt=10.0)
    
    return sol
    
    
end
    

In [ ]:
@time π_safe(state0, ϕ0)

In [ ]:
# precompile
simulate_backup_safety(state0, 15.0);

In [ ]:
sol_backup = simulate_backup_safety(state0, 50*60.0);

## Proposed Technique - Receding Horizon Backup Safety Controllers

In [ ]:
function Plots.plot!(P::Trajectory)
   
    plot!([x[1]/1000 for x in P.xs], [x[2]/1000 for x in P.xs])
    
end

In [ ]:

function generate_trajectory(t0, x0, ϕ0, T_ext, T_back)
    
    t = 1.0*t0
    x = copy(x0)
    ϕ = copy(ϕ0)
    
    dt = 1.0 # seconds
    
    P = Trajectory()
    
    while t < t0 + T_ext + T_back
        
        push!(P.ts, t)
        push!(P.xs, copy(x))

        if (t <= t0 + T_ext)
            u = π_nominal(x, ϕ)
        else
            u = π_backup(x, ϕ)
        end
        
        push!(P.us, copy(u))
        
        # propagate
        x .+= dt * (dynamics(x, u))
        
        t += dt
    end
    
    return P
    
end


function is_safe(P::Trajectory, ϕ0)
   
    t0 = P.ts[1]
    
    for (i, x) in enumerate(P.xs)
        pos = x[1:2]/1000
        dt_hours = (P.ts[i] - t0) / 3600
        
        # this is our forecast on the fire
        ϕ = ϕ0[Fire.get_index(pos)...] - max_spread_rate * dt_hours
        if ϕ < 0.0
            return false
        end
    end
    
    # check terminal
    if (P.xs[end][4] < 10.0)
        return false
    end
    
    return true
end


function π_gatekeeper(t, state, ϕ, P_com::Trajectory)
   
    if t < P_com.ts[end]
        ind = findlast(P_com.ts .<= t)
        u = P_com.us[ind]
        return u
    end
    
    # follow the backup strategy
    u = π_backup(state, ϕ)
    return u
    
end


function gatekeeper_closed_loop(state, params, t)

    P_com = params[:P_com]
    
    ϕ = Fire.get_ϕ(t/3600)
     
    u = π_gatekeeper(t, state, ϕ, P_com)
    
    return dynamics(state, u)
end


function replan!(integrator)
    
    # params = integrator.p
    
    
    
    t0 = integrator.t
    x0 = integrator.u
    ϕ0 = Fire.get_ϕ(t0/3600)
    
    
    T_ext = 20.0
    T_back = 100.0
    
    #@show "replanning at t: $(t0)"
    
    P_ext = generate_trajectory(t0, x0, ϕ0, T_ext, T_back)
    
    if is_safe(P_ext, ϕ0)
        integrator.p[:P_com] = P_ext
    else
        @show "rejecting new plan at t: $(t0)"
    end
    
end


In [ ]:
function simulate_gatekeeper(state0, t_max)
    
    tspan = (0.0, t_max)
    
    ϕ0 = Fire.get_ϕ(0.0)
    
    P_com0 = generate_trajectory(0.0, state0, ϕ0, 0.0, 120.0)
    
    @assert is_safe(P_com0, ϕ0)
    
    params = Dict(:P_com=>P_com0)
    
    
    callback = PeriodicCallback(replan!, 10.0; initial_affect=true, save_positions=(true, true)) # every 10 seconds
    
    
    prob = ODEProblem(gatekeeper_closed_loop, state0, tspan, params)
    
    sol = solve(prob, abstol=1e-2, reltol=1e-2, dt=10.0,callback=callback, dense=false)
    
    return sol
    
    
end
    

In [ ]:
@time begin
    P_ext = generate_trajectory(0.0, state0, ϕ0, 20.0, 100.0)
    is_safe(P_ext, ϕ0)
end

In [ ]:

# precompile
sol_gatekeeper = simulate_gatekeeper(state0, 25.0);


In [ ]:
sol_gatekeeper = simulate_gatekeeper(state0, 50*60)

## PLOTTING

In [ ]:
using ColorSchemes

myreds = ColorScheme([get(ColorSchemes.:linear_kry_0_97_c73_n256, f) for f in 0.25:0.01:0.75])


In [ ]:


    lay = @layout [a b c d e]

plots = []

for t=[600, 1200, 1800, 2400, 3000]
    p1 = plot()
    
    t0 = max(0.0, t-600)
    
    for t_ in t0:50:t
        xs, ys = Fire.get_ϕ_cont(t_/3600)
        f = ((t_-t0)/(t-t0))
        c = get(myreds, f)
        plot!(xs, ys, label=false, color=c, linewidth=2)
    end
    xlims!(-5,5)
    ylims!(-5,5)
    
    plot_ts = t0:1.0:t
    hxs_n = [sol(i)[1]/1000 for i in plot_ts]
    hys_n = [sol(i)[2]/1000 for i in plot_ts]

    hxs_b = [sol_backup(i)[1]/1000 for i in plot_ts]
    hys_b = [sol_backup(i)[2]/1000 for i in plot_ts]

    hxs_g = [sol_gatekeeper(i)[1]/1000 for i in plot_ts]
    hys_g = [sol_gatekeeper(i)[2]/1000 for i in plot_ts]

    
    plot!(hxs_n, hys_n, label=false, color=:black, linewidth=2)
    plot!(hxs_b, hys_b, label=false, color=:blue, linewidth=2)
    plot!(hxs_g, hys_g, label=false, color=:green, linewidth=2)
    
    plot!(ticks=nothing, grid=true, showaxis=false, link=:both, aspect_ratio=1)
    
    #plot!([0],[0], marker=:cross, label=false)
   
    push!(plots, p1)
end

plot(plots..., layout=lay)


In [ ]:
    
plots = []
begin 

    t0 =0 
    t=300
    p1 = plot()
    
    for _t in t0:5.0:t
        xs, ys = Fire.get_ϕ_cont(_t/3600)
        f = ((_t-t0)/(t-t0))
        c = get(myreds, f)
        plot!(xs, ys, label=false, color=c, linewidth=3)
    end

    xlims!(-5,5)
    ylims!(-5,5)
    
    plot_ts = t0:1.0:t
    hxs_n = [sol(i)[1]/1000 for i in plot_ts]
    hys_n = [sol(i)[2]/1000 for i in plot_ts]

    hxs_b = [sol_backup(i)[1]/1000 for i in plot_ts]
    hys_b = [sol_backup(i)[2]/1000 for i in plot_ts]

    hxs_g = [sol_gatekeeper(i)[1]/1000 for i in plot_ts]
    hys_g = [sol_gatekeeper(i)[2]/1000 for i in plot_ts]

    
    plot!(hxs_n, hys_n, label=false, color=:black, linewidth=2)
    plot!(hxs_b, hys_b, label=false, color=:blue, linewidth=2)
    plot!(hxs_g, hys_g, label=false, color=:green, linewidth=2)
    
    plot!(grid=true, showaxis=false, link=:both, aspect_ratio=1)
    
    xlims!(-0.5,4)
    ylims!(-3,1.5)
    
    push!(plots, p1)
end

plot(plots[1])

In [ ]:

    plots = []
begin 

    t0 =450
    t=t0+240
    p1 = plot()
    
    for _t in t0:5.0:t
        xs, ys = Fire.get_ϕ_cont(_t/3600)
        f = ((_t-t0)/(t-t0))
        c = get(myreds, f)
        plot!(xs, ys, label=false, color=c, linewidth=3)
    end

    xlims!(-5,5)
    ylims!(-5,5)
    
    plot_ts = t0:1.0:t
    hxs_n = [sol(i)[1]/1000 for i in plot_ts]
    hys_n = [sol(i)[2]/1000 for i in plot_ts]

    hxs_b = [sol_backup(i)[1]/1000 for i in plot_ts]
    hys_b = [sol_backup(i)[2]/1000 for i in plot_ts]

    hxs_g = [sol_gatekeeper(i)[1]/1000 for i in plot_ts]
    hys_g = [sol_gatekeeper(i)[2]/1000 for i in plot_ts]

    
    plot!(hxs_n, hys_n, label=false, color=:black, linewidth=2)
    plot!(hxs_b, hys_b, label=false, color=:blue, linewidth=2)
    plot!(hxs_g, hys_g, label=false, color=:green, linewidth=2)
    
    plot!(grid=true, showaxis=false, link=:both, aspect_ratio=1)
    
    xlims!(-0.5,4)
    ylims!(0,4.5)
    
    push!(plots, p1)
end

plot(plots[1])

In [ ]:

    plots = []
begin 

    t0 =1200
    t=t0+480
    p1 = plot()
    
    for _t in t0:5.0:t
        xs, ys = Fire.get_ϕ_cont(_t/3600)
        f = ((_t-t0)/(t-t0))
        c = get(myreds, f)
        plot!(xs, ys, label=false, color=c, linewidth=2)
    end

    xlims!(-5,5)
    ylims!(-5,5)
    
    plot_ts = t0:1.0:t
    hxs_n = [sol(i)[1]/1000 for i in plot_ts]
    hys_n = [sol(i)[2]/1000 for i in plot_ts]

    hxs_b = [sol_backup(i)[1]/1000 for i in plot_ts]
    hys_b = [sol_backup(i)[2]/1000 for i in plot_ts]

    hxs_g = [sol_gatekeeper(i)[1]/1000 for i in plot_ts]
    hys_g = [sol_gatekeeper(i)[2]/1000 for i in plot_ts]

    
    plot!(hxs_n, hys_n, label=false, color=:black, linewidth=2)
    plot!(hxs_b, hys_b, label=false, color=:blue, linewidth=2)
    plot!(hxs_g, hys_g, label=false, color=:green, linewidth=2)
    
    plot!(grid=true, showaxis=false, link=:both, aspect_ratio=1)
    
    xlims!(-0.5,4)
    ylims!(-4,0.5)
    
    push!(plots, p1)
end

plot(plots[1])

In [ ]:

    plot()
for i=0:0.005:1
    plot!([i,i],[0,1],color=get(myreds,i), label=false, linewidth=5)
end
plot!(ticks=nothing, grid=false, showaxis=false, aspect_ratio=0.1)
plot!()
savefig("colorbar.svg")
plot!()

In [ ]:

t0 = 0
t = 50*60

plot_ts = t0:5.0:t

ϕ_n = Float64[]
ϕ_b = Float64[]
ϕ_g = Float64[]

for t in plot_ts

    ϕ = Fire.get_ϕ(t/3600)
    push!(ϕ_n, ϕ[Fire.get_index(sol(t)/1000)...])
    push!(ϕ_b, ϕ[Fire.get_index(sol_backup(t)/1000)...])
    push!(ϕ_g, ϕ[Fire.get_index(sol_gatekeeper(t)/1000)...])
end


In [ ]:

plot()
plot!(plot_ts/60, ϕ_n, color=:black,linewidth=2, label="Nominal")
plot!(plot_ts/60, ϕ_b, color=:blue, linewidth=2, label = "Backup")
plot!(plot_ts/60, ϕ_g, color=:green,linewidth=2, label="Gatekeeper")
hline!([0], color=:red, label=false)
ylabel!("Dist to Fire [km]")
xlabel!("Time [min]")
plot!()
savefig("minDist.svg")
plot!()

## ANIMATION PLOTS

In [ ]:
function fire_plot!(t)
    # draw the fire
    xs, ys = Fire.get_ϕ_cont(0/3600)
    shape=Shape(xs, ys)
    plot!(shape,fill = get(myreds,0), opacity=0.2, label=false)
    for p = 0:60:t
        xs, ys = Fire.get_ϕ_cont(p/3600)
        f = p/t
        c = get(myreds, f)
        plot!(xs, ys, color=c, label=false, linewidth=3, opacity=0.5)
    end
    xs, ys = Fire.get_ϕ_cont(t/3600)
    plot!(xs, ys, color=get(myreds,1), label="firefront")
end

In [ ]:
function pretty_plot!(t)
    

    fire_plot!(t)
    
    plot_ts = max(0.0, t-5*60.0):1.0:t
    hxs_n = [sol(i)[1]/1000 for i in plot_ts];
    hys_n = [sol(i)[2]/1000 for i in plot_ts];
    hxs_b = [sol_backup(i)[1]/1000 for i in plot_ts];
    hys_b = [sol_backup(i)[2]/1000 for i in plot_ts];
    hxs_g = [sol_gatekeeper(i)[1]/1000 for i in plot_ts];
    hys_g = [sol_gatekeeper(i)[2]/1000 for i in plot_ts];

    plot!(hxs_n, hys_n; color=:black, linewidth=(3*[i/length(plot_ts) for i=1:length(plot_ts)]), linestyle=:dash, label="nominal")
    plot!(hxs_b, hys_b; color=:blue, linewidth=(3*[i/length(plot_ts) for i=1:length(plot_ts)]), label="backup filter")
    plot!(hxs_g, hys_g; color=:green, linewidth=(3*[i/length(plot_ts) for i=1:length(plot_ts)]), label="gatekeeper")
    xlims!(-6,6)
    ylims!(-6,6)
    plot!(aspect_ratio=1, legend = :outertopright, framestyle = :box)

    
    return 
end

In [ ]:
n = 960
# i=5
# begin
anim = @animate for i ∈ 1:n
    
    l = @layout [
    a{0.7w} [b{0.5h}
             c{0.4h}  ]
    ]

    
    p1 = plot()
    
    t = (i/n) * (50 * 60)
    pretty_plot!(t)
        xlabel!("x [km]")
    ylabel!("y [km]")
    plot!(legend=:topleft,  foreground_color_legend = nothing)
    

    p2 = plot()
    #, inset = (1, bbox(0.75, 0.65, 0.25, 0.35))
    hx = 0.5 * (sol(t)[1] + sol_gatekeeper(t)[1])/1000 
    hy = 0.5 * (sol(t)[2] + sol_gatekeeper(t)[2])/1000 
    pretty_plot!(t)
    plot!(ticks=nothing, legend=false)
    xlims!(hx-1.5, hx+1.5)
    ylims!(hy-1.5, hy+1.5)
#     plot!(showaxis=:off)
#     ylabel!("")
    
    p3 = plot()
    ϕ = Fire.get_ϕ(t/3600)
    ϕ_n = ϕ[Fire.get_index(sol(t)/1000)...]
    ϕ_b = ϕ[Fire.get_index(sol_backup(t)/1000)...]
    ϕ_g = ϕ[Fire.get_index(sol_gatekeeper(t)/1000)...]
    
    cs = [
        (ϕ_n <= 0.0 ? :red : :black)
        (ϕ_b <= 0.0 ? :red : :blue)
        (ϕ_g <= 0.0 ? :red : :green)
    ]
    
    bar!(["nominal", "backup filter", "gatekeeper"], [ϕ_n, ϕ_b, ϕ_g], legend=false, color=cs)
    plot!(xrotation  = 45.0)
    hline!([0.0], color=:red)
    hline!([0.1], color=:gray, linestyle=:dash)
    ylabel!("Dist to Fire [km]")
    plot!(yguidefontsize=10)
    ylims!(-0.25,1.0)
    yticks!([0,1.0])
    plot(p1, p2, p3, layout=l)
end
# gif(anim, "anim_fps15.gif", fps = 15)



In [ ]:
gif(anim, "anim_fps30.gif", fps = 30)

In [ ]:
gif(anim, "anim_fps15.gif", fps = 15)

In [ ]:
n = 240
# i=5
# begin
anim = @animate for i ∈ 1:n
    
    p1 = plot()
    
    t = (i/n) * (50 * 60)
    fire_plot!(t)
    plot!(legend=false,  foreground_color_legend = nothing)
    xlims!(-6,6)
    ylims!(-6,6)
    plot!(ticks=false)
    plot!(aspect_ratio=1, framestyle = :none)
    
end
# gif(anim, "anim_fps15.gif", fps = 15)



In [ ]:
gif(anim, "anim_fire_only_30fps.gif", fps = 30)